<a href="https://colab.research.google.com/github/I-scream-icecream/Finance-dashboard/blob/main/Forexfactory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium streamlit pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [3]:
import random
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import csv

def create_driver():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    ]
    user_agent = random.choice(user_agent_list)

    browser_options = webdriver.ChromeOptions()
    browser_options.add_argument("--no-sandbox")
    browser_options.add_argument("--headless")
    browser_options.add_argument("start-maximized")
    browser_options.add_argument("window-size=1900,6000")
    browser_options.add_argument("disable-gpu")
    browser_options.add_argument("--disable-software-rasterizer")
    browser_options.add_argument("--disable-dev-shm-usage")
    browser_options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(options=browser_options)

    return driver

def parse_data(driver, url):
    driver.get(url)

    data_table = driver.find_element(By.CLASS_NAME, "calendar__table")
    value_list = []

    for row in data_table.find_elements(By.TAG_NAME, "tr"):
        row_data = list(filter(None, [td.text for td in row.find_elements(By.TAG_NAME, "td")]))
        if row_data:
            value_list.append(row_data)  # Append only if row_data has content

    # Corrected variable name for consistency
    result_list = []
    currencies = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
    for row in value_list:
        if len(row) == 1:
            result_list.append(row)
        elif row[1] in currencies:
            result_list.append([None] + row)  # Add None for missing Date
        elif row[0] in currencies:
            result_list.append([None, None] + row)  # Add None for both Date and Time
        else:
            result_list.append(row)  # Keep the row as is

    # Create DataFrame with appropriate column names
    df = pd.DataFrame(result_list, columns=['Date', 'Time', 'Currency', 'Impact', 'Actual', 'Forecast', 'Previous'])

    # Save DataFrame to CSV
    df.to_csv("table_data.csv", index=False)
    print("done")

driver = create_driver()
url = 'https://www.forexfactory.com/calendar'

value_list = parse_data(driver=driver, url=url)



done


V2 code with download button and streamlit deploy

In [4]:
%%writefile forexfactory.py

import base64
import os
import streamlit as st
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def create_driver():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    ]
    user_agent = random.choice(user_agent_list)

    browser_options = webdriver.ChromeOptions()
    browser_options.add_argument("--no-sandbox")
    browser_options.add_argument("--headless")
    browser_options.add_argument("start-maximized")
    browser_options.add_argument("window-size=1900,6000")
    browser_options.add_argument("disable-gpu")
    browser_options.add_argument("--disable-software-rasterizer")
    browser_options.add_argument("--disable-dev-shm-usage")
    browser_options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(options=browser_options)

    return driver

def parse_data(driver, url):
    driver.get(url)

    data_table = driver.find_element(By.CLASS_NAME, "calendar__table")
    value_list = []

    for row in data_table.find_elements(By.TAG_NAME, "tr"):
        row_data = list(filter(None, [td.text for td in row.find_elements(By.TAG_NAME, "td")]))
        if row_data:
            value_list.append(row_data)  # Append only if row_data has content

    # Corrected variable name for consistency
    result_list = []
    currencies = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
    for row in value_list:
        if len(row) == 1:
            result_list.append(row)
        elif row[1] in currencies:
            result_list.append([None] + row)  # Add None for missing Date
        elif row[0] in currencies:
            result_list.append([None, None] + row)  # Add None for both Date and Time
        else:
            result_list.append(row)  # Keep the row as is

    # Create DataFrame with appropriate column names
    df = pd.DataFrame(result_list, columns=['Date', 'Time', 'Currency', 'Impact', 'Actual', 'Forecast', 'Previous'])

    # Save DataFrame to CSV
    df.to_csv("table_data.csv", index=False)
    st.write("CSV file generated successfully!")

def get_binary_file_downloader_html(bin_file, file_label='File'):
    with open(bin_file, 'rb') as f:
        data = f.read()
    b64_data = base64.b64encode(data).decode()
    href = f'<a href="data:file/csv;base64,{b64_data}" download="{bin_file}">{file_label}</a>'
    return href

# Streamlit UI
st.title('ForexFactory Web Scraper')

if st.button('Scrape Data'):
    with st.spinner('Scraping data...'):
        driver = create_driver()
        url = 'https://www.forexfactory.com/calendar'
        parse_data(driver=driver, url=url)

        # Display CSV file and provide download link
        if os.path.exists("/content/table_data.csv"):
            st.subheader("CSV file generated:")
            df = pd.read_csv("table_data.csv")
            st.write(df)

            st.markdown(get_binary_file_downloader_html("table_data.csv", 'CSV file'), unsafe_allow_html=True)




Writing forexfactory.py


In [5]:
! wget -q -O - ipv4.icanhazip.com

34.75.80.148


In [ ]:
! streamlit run forexfactory.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.80.148:8501

npx: installed 22 in 4.978s
your url is: https://dark-hats-find.loca.lt
